In [5]:
import numpy as np
from scipy.sparse import spdiags, kron, eye

# Parameters
L = 10  # Domain [-L, L]
n = 8   # Number of grid points
dx = 2 * L / n
x = np.linspace(-L, L - dx, n)

# Identity matrix
I = eye(n, format='csr')

# First derivative matrix (∂x or ∂y)
e = np.ones(n)
diagonals_D1 = [-e, e]
offsets_D1 = [-1, 1]
D1 = spdiags(diagonals_D1, offsets_D1, n, n) / (2 * dx)
D1 = D1.tolil()  # Convert to modify boundary conditions
D1[0, -1] = -1 / (2 * dx)  # Periodic condition
D1[-1, 0] = 1 / (2 * dx)   # Periodic condition
D1 = D1.tocsr()  # Convert back to sparse CSR format

# Second derivative matrix (∂²x or ∂²y)
diagonals_D2 = [e, -2 * e, e]
offsets_D2 = [-1, 0, 1]
D2 = spdiags(diagonals_D2, offsets_D2, n, n) / dx**2
D2 = D2.tolil()  # Convert to modify boundary conditions
D2[0, -1] = 1 / dx**2  # Periodic condition
D2[-1, 0] = 1 / dx**2  # Periodic condition
D2 = D2.tocsr()  # Convert back to sparse CSR format

# Construct 2D matrices using Kronecker products
I_n = eye(n, format='csr')  # Identity matrix for grid size
D1x = kron(D1, I_n, format='csr')  # ∂x (size: n^2 x n^2)
D1y = kron(I_n, D1, format='csr')  # ∂y (size: n^2 x n^2)
D2x = kron(D2, I_n, format='csr')  # ∂²x (size: n^2 x n^2)
D2y = kron(I_n, D2, format='csr')  # ∂²y (size: n^2 x n^2)

# Final matrices
A1 = D2x + D2y  # Laplacian (size: n^2 x n^2)
A2 = D1x        # ∂x (size: n^2 x n^2)
A3 = D1y        # ∂y (size: n^2 x n^2)

# Convert to dense matrices for submission
A1 = A1.toarray()
A2 = A2.toarray()
A3 = A3.toarray()

# Output
print("Matrix A1 (Laplacian): Shape", A1)
print("Matrix A2 (∂x): Shape", A2)
print("Matrix A3 (∂y): Shape", A3)

Matrix A1 (Laplacian): Shape [[-0.64  0.16  0.   ...  0.    0.    0.  ]
 [ 0.16 -0.64  0.16 ...  0.    0.    0.  ]
 [ 0.    0.16 -0.64 ...  0.    0.    0.  ]
 ...
 [ 0.    0.    0.   ... -0.64  0.16  0.  ]
 [ 0.    0.    0.   ...  0.16 -0.64  0.16]
 [ 0.    0.    0.   ...  0.    0.16 -0.64]]
Matrix A2 (∂x): Shape [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Matrix A3 (∂y): Shape [[ 0.   0.2  0.  ...  0.   0.   0. ]
 [-0.2  0.   0.2 ...  0.   0.   0. ]
 [ 0.  -0.2  0.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  0.   0.2  0. ]
 [ 0.   0.   0.  ... -0.2  0.   0.2]
 [ 0.   0.   0.  ...  0.  -0.2  0. ]]
